# **Define Intents**

In [2]:
import os
os.system('mkdir /content/sample_data/chatbot')

0

In [3]:
nlu_md = """

## intent:greet
- hey
- hello
- hi
- Hi

## intent:fine_ask
- I am good, how are you doing?
- I'm fine, how are you?
- I'm good, how are you?

## intent:my_name_is
- I am [Atul](PERSON)
- I am [Sampriti](PERSON)
- I'm [Prerna](PERSON)
- im [Varun](PERSON)
- My name is [Nikhil](PERSON)

## intent:fine_normal
- I am doing great
- I'm doing great
- I'm fine
- I'm good

## intent:news
- Share some latest news around the [world](category)?
- Share some latest news in [sports](category)?
- What is going on in [technology](category)?
- Tell me some news about [fashion](category)
- Tell me some news about [business](category)
- Tell me some news about [arts](category)
- What is going on in [arts](category)
- What is cooking in [food](category)
- [movies](category)

## intent:thanks
- Thanks
- Thank you so much

## intent:bye
- No, I am good as of now. Bye
- Bye
- Bbye
"""


f = open('/content/sample_data/chatbot/nlu.md', 'w')
f.write(nlu_md)
f.close()

# **NLU configuration and Policy definition**

In [4]:
config = """
language: "en"

pipeline:
- name: "WhitespaceTokenizer"
- name: "RegexFeaturizer"
- name: "CRFEntityExtractor"
- name: "EntitySynonymMapper"
- name: "CountVectorsFeaturizer"
- name: "CountVectorsFeaturizer"
  analyzer: "char_wb"
  min_ngram: 1
  max_ngram: 4
- name: "EmbeddingIntentClassifier"

policies:
  - name: KerasPolicy
    epochs: 100
    max_history: 3
  - name: MemoizationPolicy
    max_history: 3
  - name: FallbackPolicy
    nlu_threshold: 0.1
    core_threshold: 0.2
    fallback_action_name: 'utter_default'
  - name: FormPolicy

""" 

f = open('/content/sample_data/chatbot/config.yml', 'w')
f.write(config)
f.close()

# **Defining stories**

In [5]:
stories_md = """
## fallback
- utter_default

## greeting path 1
* greet
- utter_greet

## my name is
* my_name_is
- utter_its_nice_to_meet_you

## fine path 1
* fine_normal
- utter_help

## fine path 2
* fine_ask
- utter_reply

## news path
* news
- action_get_news
- utter_ofc

## thanks path 1
* thanks
- utter_anything_else

## bye path 1
* bye
- utter_bye
"""

f = open('/content/sample_data/chatbot/stories.md', 'w')
f.write(stories_md)
f.close()

# **Define Domain**

In [6]:
domain_yml = """
slots:
  category:
    type: text
  PERSON:
    type: text

entities:
- category
- PERSON

intents:
- greet
- my_name_is
- fine_ask
- fine_normal
- news
- thanks
- bye

actions:
- action_restart
- action_get_news
- utter_greet
- utter_reply
- utter_help
- utter_anything_else
- utter_ofc
- utter_bye
- utter_default
- utter_its_nice_to_meet_you

templates:

  utter_greet:
    - text: Hey {PERSON}, how are you?
    - text: Hello {PERSON}, How are you doing?
    
  utter_its_nice_to_meet_you:
    - It's nice to meet you, {PERSON}.
    - Nice to meet you, {PERSON}.
  
  utter_reply:
    - text: I'm doing great. Please let me know what I can do for you.
    - text: I'm doing great. Tell me How can I help you today?
  utter_help:
    - text: Great{PERSON}. How can I help you?
    - text: Great. Tell me How can I help you?
    - text: Great. Tell me what all news you would like to get.
  utter_anything_else:
    - text: No worries. Is there anything else I can help you with?
    - text: No worries. Let me know if there is anything else I can help you with
  utter_ofc:
    - text: I can definitely help you. The top 5 news of the {category}
    - text: Surely, I can help you. The top 5 news of the {category}
  utter_bye:
    - text: Bye and have a nice day
    - text: Bbye and have a nice day
  utter_default:
    - text: I am not sure what you're aiming for
    - text: I am sorry but I am not able to get you.
    - text: My appologies but I am not able to get you
"""

f = open('/content/sample_data/chatbot/domain.yml', 'w')
f.write(domain_yml)
f.close()

# **Defining custom action as python code in actions.py**

In [7]:
custom_action = """


from typing import Text, List, Dict, Any
from rasa_sdk import Action, Tracker
from rasa_sdk.events import SlotSet, SessionStarted, ActionExecuted, EventType
from rasa_sdk.executor import CollectingDispatcher
import requests
import json

import requests

class ApiAction(Action):
    def name(self):
        return "action_get_news"

    def run(self,
           dispatcher: CollectingDispatcher,
           tracker: Tracker,
           domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:

        category = tracker.get_slot('category')
        print(category)
        
        url = 'https://api.nytimes.com/svc/news/v3/content/all/{category}.json'.format(category=category)
        params = {'api-key': "2hq54bvFO0yWiRdY70reBU2GmusBtnwM", 'limit': 5}
        response = requests.get(url, params).text
        json_data = json.loads(response)['results']
        i = 0
        for results in json_data:
            i = i+1
            message = str(i) + "." + results['abstract']
            dispatcher.utter_message(message)
        return[]
        
"""
f = open('/content/sample_data/chatbot/actions.py', 'w')
f.write(custom_action)
f.close()

# **Building the BOT**

In [8]:
import nest_asyncio
nest_asyncio.apply()
print("Event loop ready.")

Event loop ready.


In [9]:
import logging, io, json, warnings
# logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

In [11]:
!pip install rasa==1.10.12

     |████████████████████████████████| 512 kB 5.1 MB/s 
     |████████████████████████████████| 3.0 MB 10.2 MB/s 
     |████████████████████████████████| 1.6 MB 40.1 MB/s 
     |████████████████████████████████| 148 kB 50.8 MB/s 
     |████████████████████████████████| 990 kB 60.3 MB/s 
     |████████████████████████████████| 104 kB 51.6 MB/s 
     |████████████████████████████████| 3.2 MB 60.6 MB/s 
     |████████████████████████████████| 56 kB 3.7 MB/s 
     |████████████████████████████████| 49 kB 6.0 MB/s 
     |████████████████████████████████| 101 kB 11.4 MB/s 
     |████████████████████████████████| 174 kB 49.1 MB/s 
     |████████████████████████████████| 96 kB 6.0 MB/s 
     |████████████████████████████████| 375 kB 42.6 MB/s 
     |████████████████████████████████| 40 kB 5.2 MB/s 
     |████████████████████████████████| 131 kB 47.5 MB/s 
     |████████████████████████████████| 149 kB 46.4 MB/s 
     |████████████████████████████████| 111 kB 48.2 MB/s 
     |█████████████████

In [12]:
import spacy
print(spacy.load("en")("Hello world!"))

Hello world!


In [13]:
configfile = "/content/sample_data/chatbot/config.yml"
training_files = "/content/sample_data/chatbot"
domainfile = "/content/sample_data/chatbot/domain.yml"
outputdir = "/content/sample_data/chatbot/models"
print("config:",configfile)
print("nlu.md,story.md:", training_files)
print("domain:", domainfile)
print("models:", outputdir)

config: /content/sample_data/chatbot/config.yml
nlu.md,story.md: /content/sample_data/chatbot
domain: /content/sample_data/chatbot/domain.yml
models: /content/sample_data/chatbot/models


In [16]:
import rasa
model_path = rasa.train(domainfile, configfile, [training_files], outputdir)
print(model_path)

Training Core model...


Processed trackers: 100%|██████████| 508/508 [00:08<00:00, 63.21it/s, # actions=219]


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 3, 29)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                7936      
_________________________________________________________________
dense (Dense)                (None, 18)                594       
_________________________________________________________________
activation (Activation)      (None, 18)                0         
Total params: 8,530
Trainable params: 8,530
Non-trainable params: 0
_________________________________________________________________
Train on 219 samples
Epoch 1/100
219/219 [==============================] - 6s 28ms/sample - loss: 2.8505 - accuracy: 0.2237
Epoch 2/100
219/219 [==============================] - 0s 249us/sample - loss: 2.7784 - accuracy: 0.3790
Epoch 3/100
219/219 [=

Processed trackers: 100%|██████████| 8/8 [00:00<00:00, 159.58it/s, # actions=17]
Processed actions: 17it [00:00, 401.19it/s, # examples=17]
Processed trackers: 100%|██████████| 8/8 [00:00<00:00, 173.88it/s, # actions=17]


Core model training completed.
Training NLU model...


Epochs: 100%|██████████| 300/300 [00:17<00:00, 16.99it/s, t_loss=1.509, i_loss=0.660, i_acc=1.000]


NLU model training completed.
Your Rasa model is trained and saved at '/content/sample_data/chatbot/models/20210721-105214.tar.gz'.
/content/sample_data/chatbot/models/20210721-105214.tar.gz


In [17]:
from rasa.jupyter import chat
chat(model_path)

Your bot is ready to talk! Type your messages here or send '/stop'.
Hi. I am Atul
It's nice to meet you, Atul.
How are you
I'm doing great. Tell me How can I help you today?
Give me some news on world
Surely, I can help you. The top 5 news of the world
/stop
